In [1]:
import pandas as pd
import ast

In [2]:
RAW_DATA_PATH: str = 'data/dataset.raw.csv'
EXPORTED_DATA_PATH: str = 'data/dataset'

TRAIN_SPLIT: float = 0.8
RANDOM_STATE: int = 42

In [3]:
raw: pd.DataFrame = (
    pd.read_csv(
        RAW_DATA_PATH,
        skiprows=[1],
        index_col=[0],
        dtype={
            'text': 'string',
            'label': 'category'
        },
        converters={
            'span': ast.literal_eval
        }
    )
    .rename_axis('idx')
)
raw

,text,label,span
idx,,,
c4f6dccf-dc39-40cc-a6c0-1eeec9fc0ab5,Diese Maske ist nicht nur für medizinisches Pe...,corona,"[6, 11]"
f6310dbb-e8fa-499c-9835-a85daf81bdda,Je nach Antwort auf diese Frage sind die Mensc...,corona,"[115, 120]"
2ffbedf4-b35a-4234-bacf-cba2dd6154d2,"Man trägt halt eine Maske, hält Distanz, und d...",corona,"[20, 25]"
edae875f-1bbc-449c-b706-b983ed261b63,"Ein Link führt zum Webshop, dort kostet die Pa...",corona,"[60, 65]"
6e27b81a-c66f-4ea6-982b-b844a428db2c,"„Ich bin da sehr dankbar, wenn Bußgelder verhä...",corona,"[86, 91]"
...,...,...,...
a77612b9-19ba-461a-a9b3-955b447d4723,Nach Apfel riecht die Maske jedoch nicht.,web1,"[22, 27]"
da99e86c-b2ce-41e2-98ec-53e812262d9f,1. Wasche Dein Haar mit einem geeigneten Shamp...,web1,"[101, 106]"
7eb73a31-0b09-4286-913e-48861a4f97cd,Die Lieferung beinhaltet 10 Masken.,web1,"[28, 33]"


In [4]:
raw.dtypes

text     string[python]
label          category
span             object
dtype: object

In [5]:
raw.label.unique()

['corona', 'web2', 'web1']
Categories (3, object): ['corona', 'web1', 'web2']

In [6]:
raw.label.value_counts()

label
corona    4983
web1      4928
web2      4873
Name: count, dtype: int64

In [7]:
(
    raw
    .apply(lambda row: row.text[row.span[0]:row.span[1]], axis=1)
    .value_counts()
)

Maske    14784
Name: count, dtype: int64

In [8]:
train_split = raw.sample(
    frac=TRAIN_SPLIT, 
    random_state=RANDOM_STATE
)
train_split.to_parquet(f'{EXPORTED_DATA_PATH}/train.parquet')

In [9]:
test_split = raw.loc[
    raw.index.difference(train_split.index)
]
test_split.to_parquet(f'{EXPORTED_DATA_PATH}/test.parquet')